In [96]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException
import pandas as pd
import datetime
import time
from itertools import zip_longest
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe 
import seaborn as sns
import googlemaps
import requests
from geopy.geocoders import Nominatim
import folium
from IPython.display import IFrame


In [97]:
navegador = webdriver.Chrome()  ## iniciando o webdriver do navegador

todos_os_dados = []  ## criando lista vazia para armazenar os dados de diferentes pa´ginas

pagina = 1  ## contador de páginas sendo atribuído para 1
while True:
    url = f"https://www.olx.com.br/imoveis/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?o={pagina}"  ## esse valor entre páginas serve para identificar em qual página o WebDriver está lendo e por ser um contador, a cada loop uma nova página é lida
    print(f"\nAcessando página {pagina}: {url}")
    navegador.get(url)

    try:
        WebDriverWait(navegador, 12).until(ec.presence_of_all_elements_located((
            By.CSS_SELECTOR, 'h3.olx-text.olx-text--body-large.olx-text--block.olx-text--semibold.olx-adcard__price'
        )))  ## Adicionando comando de espera para garantir que todos os elementos da página foram devidamente carregados

        elementos_valores = navegador.find_elements(By.CSS_SELECTOR, 'h3.olx-text.olx-text--body-large.olx-text--block.olx-text--semibold.olx-adcard__price')   ## localizando os elementos de preco
        elementos_localizacoes = navegador.find_elements(By.CSS_SELECTOR, 'p.olx-adcard__location')   ## localizando os elementos de localização
        elementos_quartos = navegador.find_elements(By.CSS_SELECTOR, 'h3.olx-text.olx-text--body-large.olx-text--block.olx-text--semibold.olx-adcard__price')   ## localizando os elementos de preco
        elementos_localizacoes = navegador.find_elements(By.CSS_SELECTOR, 'p.olx-adcard__location')   ## localizando os elementos de localização
        detalhes = navegador.find_elements(By.CLASS_NAME, 'olx-adcard__detail')

        lista_valores = [valor.text.strip() for valor in elementos_valores]  ## gerando a lista com os valores dos imóveis de uma página
        lista_localizacoes = [loc.text.strip() for loc in elementos_localizacoes] ## gerando a lista com as localizações dos imóveis de uma página
        lista_quartos = [loc.text.strip() for loc in elementos_localizacoes] ## gerando a lista com as localizações dos imóveis de uma página
        lista_areas = []
        for detalhe in detalhes:
            texto = detalhe.text.strip()
            if 'm²' in texto:
                lista_areas.append(texto)
        

        if not lista_valores:  ## se não há valores na página, não ocorre raspagem
            print("Sem valores nesta página. Fim da raspagem.")
            break

        df_valores = pd.DataFrame(
            list(zip_longest(lista_valores, lista_localizacoes, lista_areas, fillvalue="")), ## garantindo que os espaços vazios nas lista sejam preenchidos com ""
            columns=["valor", "localizacao", "m²"])
        df_valores["data"] = datetime.date.today()  ## gerando coluna com a data de extração dos registros

        todos_os_dados.append(df_valores)  ## juntando todos os dados extraídos
        print(f"Total extraído nesta página: {len(df_valores)}")

        pagina += 1  ## a cada loop, o contador de página assume +1 e a url é alterada, fazendo com que uma nova página seja lida
        time.sleep(2) ## aguarda 2 segundos para começar o novo loop

    except TimeoutException:
        print("Nenhum anúncio encontrado ou página demorou a carregar. Encerrando.")
        break   ## se ocorrer o erro Timeouexcpetion, o loop é encerrado

df_final = pd.concat(todos_os_dados, ignore_index=True)  ## juntando todos os dados obtidos de todas as páginas lidas
print(f"\nTotal geral de valores extraídos: {len(df_final)}")
print(df_final)

navegador.quit()



Acessando página 1: https://www.olx.com.br/imoveis/estado-rj/serra-angra-dos-reis-e-regiao/petropolis?o=1


KeyboardInterrupt: 

In [99]:
lista_areas

['221m²',
 '212m²',
 '350m²',
 '339m²',
 '204m²',
 '2200m²',
 '190m²',
 '30m²',
 '69m²',
 '90m²',
 '1278m²',
 '1814m²',
 '20950m²',
 '305m²',
 '7783m²',
 '2538m²',
 '512m²',
 '430m²',
 '18m²',
 '27m²',
 '420m²',
 '250m²',
 '450m²',
 '500m²',
 '9180m²',
 '80m²',
 '300m²',
 '27m²',
 '90m²',
 '117m²',
 '147m²',
 '67m²',
 '220m²',
 '150m²',
 '84m²',
 '5000m²',
 '2500m²',
 '150m²',
 '89m²',
 '400m²',
 '75m²',
 '650m²',
 '109m²',
 '350m²',
 '450m²',
 '79m²',
 '239m²',
 '2200m²',
 '84m²',
 '171m²']

In [ ]:
df_final[['cidade','bairro']] = df_final['localizacao'].str.split(", ", n=1, expand=True)
df_final = df_final.drop(columns='localizacao')

In [ ]:
df_final.isnull().sum()

valor     0
m²        0
data      0
cidade    0
bairro    2
dtype: int64

In [98]:
df_final.dropna(subset='bairro', inplace=True)

In [101]:
index_drop = df_final.loc[df_final['m²'] == ""].index
df_final = df_final.drop(index_drop)

In [102]:
df_final['concat_local'] = df_final['cidade'] + ", " + df_final['bairro']
df_final

,valor,m²,data,cidade,bairro,concat_local
0,R$ 2.500,1000m²,2025-09-14,Petrópolis,Coronel Veiga,"Petrópolis, Coronel Veiga"
1,R$ 430.000,56m²,2025-09-14,Petrópolis,Corrêas,"Petrópolis, Corrêas"
2,R$ 2.500,59m²,2025-09-14,Petrópolis,Itaipava,"Petrópolis, Itaipava"
3,R$ 420.000,60m²,2025-09-14,Petrópolis,Corrêas,"Petrópolis, Corrêas"
4,R$ 305.000,1040m²,2025-09-14,Petrópolis,Bingen,"Petrópolis, Bingen"
...,...,...,...,...,...,...
4995,R$ 2.000,79m²,2025-09-14,Petrópolis,Bingen,"Petrópolis, Bingen"
4996,R$ 495.000,239m²,2025-09-14,Petrópolis,Independência,"Petrópolis, Independência"
4997,R$ 6.000,2200m²,2025-09-14,Petrópolis,Pedro do Rio,"Petrópolis, Pedro do Rio"
4998,R$ 1.100.000,84m²,2025-09-14,Petrópolis,Itaipava,"Petrópolis, Itaipava"


In [104]:
df_final['concat_local'].unique()

array(['Petrópolis, Coronel Veiga', 'Petrópolis, Corrêas',
       'Petrópolis, Itaipava', 'Petrópolis, Bingen', 'Petrópolis, Centro',
       'Petrópolis, Castelanea', 'Petrópolis, Quissama',
       'Petrópolis, Samambaia', 'Petrópolis, Mosela',
       'Petrópolis, Nogueira', 'Petrópolis, Pedro do Rio',
       'Petrópolis, Cuiabá', 'Petrópolis, Araras',
       'Petrópolis, Quitandinha', 'Petrópolis, Independência',
       'Petrópolis, São Sebastião', 'Petrópolis, Alto da Serra',
       'Petrópolis, Retiro', 'Petrópolis, Quarteirão Brasileiro',
       'Petrópolis, Saldanha Marinho', 'Petrópolis, Roseiral',
       'Petrópolis, Carangola', 'Petrópolis, Vila Militar',
       'Petrópolis, Chácara Flora', 'Petrópolis, Posse',
       'Petrópolis, Bonsucesso', 'Petrópolis, Simeria',
       'Petrópolis, Duarte Silveira', 'Petrópolis, Morin',
       'Petrópolis, Quarteirão Ingelheim',
       'Petrópolis, Estrada da Saudade', 'Petrópolis, Fazenda Inglesa',
       'Petrópolis, Gloria', 'Petrópolis,

In [ ]:
def aplica_latlong (lista):
  locator = Nominatim(user_agent="name_of_your_app")
  lat = []
  long = []
  j = 1
  for i in lista:
    loc = locator.geocode(i)
    print(f"{j} - {i} - {loc.latitude}, {loc.longitude}\n")
    lat.append(loc.latitude)
    long.append(loc.longitude)
    latitude = pd.DataFrame(lat)
    latitude.rename(columns = {0:"latitude"}, inplace = True)
    longitude = pd.DataFrame(long)
    longitude.rename(columns = {0:"longitude"}, inplace=True)
    coord = pd.concat([latitude, longitude], axis = 1)
    j += 1

  return coord

In [ ]:
coord = pd.DataFrame(aplica_latlong(df_final['concat_local'].unique()))
##df_final = pd.concat([df_final, coord], axis=1)

1 - Petrópolis, Coronel Veiga - -22.5262637, -43.194226

2 - Petrópolis, Corrêas - -22.4425647, -43.1397058

Valor nulo encontrado!continuando...
1 - Petrópolis, Coronel Veiga - -22.5262637, -43.194226

2 - Petrópolis, Corrêas - -22.4425647, -43.1397058

Valor nulo encontrado!continuando...
1 - Petrópolis, Coronel Veiga - -22.5262637, -43.194226

2 - Petrópolis, Corrêas - -22.4425647, -43.1397058

Valor nulo encontrado!continuando...
1 - Petrópolis, Coronel Veiga - -22.5262637, -43.194226

2 - Petrópolis, Corrêas - -22.4425647, -43.1397058

Valor nulo encontrado!continuando...
1 - Petrópolis, Coronel Veiga - -22.5262637, -43.194226

2 - Petrópolis, Corrêas - -22.4425647, -43.1397058

Valor nulo encontrado!continuando...
Valor nulo encontrado!continuando...
1 - Petrópolis, Coronel Veiga - -22.5262637, -43.194226

2 - Petrópolis, Corrêas - -22.4425647, -43.1397058

Valor nulo encontrado!continuando...
1 - Petrópolis, Coronel Veiga - -22.5262637, -43.194226

2 - Petrópolis, Corrêas - -22.

In [110]:
coord

,latitude,longitude
0,-22.526264,-43.194226
1,-22.442565,-43.139706
2,-22.412700,-43.142100
3,-22.513502,-43.214946
4,-22.509981,-43.175489
5,-22.528498,-43.189943
6,-22.492869,-43.159080
7,-22.462500,-43.130554
8,-22.493190,-43.202826
9,-22.422239,-43.130884


In [ ]:
mapa_imoveis = folium.Map(location=[-22.5099814, -43.1754892])
for i, row in df_final.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']]
    ).add_to(mapa_imoveis)

display(mapa_imoveis)